In [163]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from google.cloud import storage
import gcsfs
from shapely.geometry import Polygon
import decimal

In [61]:
pd.options.display.max_columns=None

In [2]:
credentials = '../robin-sandpit-f5abc8fc3d93.json'

In [3]:
engine = create_engine('bigquery://robin-sandpit', credentials_path=credentials)

In [6]:
file_path = 'gs://rd_experiments/mel_estate.csv'

In [7]:
df = pd.read_csv(file_path, storage_options={"token": credentials})

### 1. Upload dataframe as common table i.e., without geospatial column

In [36]:
df.to_gbq('melbourne_real_estate.mel_estate_data_til_2017', project_id='robin-sandpit')

### 2. Upload geoDataFrame to BQ table i.e., including geospatial column

In [8]:
df.head(2)

,ListingId,Agency,Price,DateSold,StreetAddress,Postcode,Locality,Latitude,Longitude,LandSize,...,1SecondaryDistance,1SecondaryScore,2SecondaryName,2SecondarySector,2SecondaryDistance,2SecondaryScore,3SecondaryName,3SecondarySector,3SecondaryDistance,3SecondaryScore
0,125268674,First National Real Estate Mike Brown - CHIRNS...,9500000,2017-06-16,111 Mangans Rd,3140,Lilydale,-37.754930,145.371809,33285,...,1281.292691,79,Lilydale High School,Government,2400.727151,74,Lilydale Heights College,Government,2990.908676,67
1,125255346,Marshall White - Stonnington,8150000,2017-05-23,7 Dunraven Avenue,3142,Toorak,-37.845278,145.014403,634,...,984.079139,89,Lauriston Girls' School,Independent,1231.496278,99,St Kevin's College,Catholic,1385.246314,97


In [27]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs="EPSG:4326")

In [28]:
gdf.head(2)

,ListingId,Agency,Price,DateSold,StreetAddress,Postcode,Locality,Latitude,Longitude,LandSize,...,1SecondaryScore,2SecondaryName,2SecondarySector,2SecondaryDistance,2SecondaryScore,3SecondaryName,3SecondarySector,3SecondaryDistance,3SecondaryScore,geometry
0,125268674,First National Real Estate Mike Brown - CHIRNS...,9500000,2017-06-16,111 Mangans Rd,3140,Lilydale,-37.754930,145.371809,33285,...,79,Lilydale High School,Government,2400.727151,74,Lilydale Heights College,Government,2990.908676,67,POINT (145.37181 -37.75493)
1,125255346,Marshall White - Stonnington,8150000,2017-05-23,7 Dunraven Avenue,3142,Toorak,-37.845278,145.014403,634,...,89,Lauriston Girls' School,Independent,1231.496278,99,St Kevin's College,Catholic,1385.246314,97,POINT (145.01440 -37.84528)


In [30]:
gdf['geometry'] = gdf['geometry'].to_wkt()

/home/robin/Storage/miniconda3/envs/gcp/lib/python3.10/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [34]:
type_dict = {
    'b' : 'BOOLEAN',
    'i' : 'INTEGER',
    'f' : 'FLOAT',
    'O' : 'STRING',
    'S' : 'STRING',
    'U' : 'STRING'
}

In [39]:
schema = [{'name' : col_name, 'type' : "GEOGRAPHY" if col_name == "geometry" else type_dict.get(col_type.kind, 'STRING')} for (col_name, col_type) in gdf.dtypes.items()]

In [42]:
gdf.to_gbq('melbourne_real_estate.mel_estate_data_til_2017_geo', project_id='robin-sandpit', table_schema=schema)

### 3. Load the geoPolygon dataset

In [121]:
df = pd.read_json('gs://rd_experiments/school_zone.json', storage_options={"token": credentials})

In [122]:
df = pd.json_normalize(df['schools'])

##### 3.1 Primary schools

In [123]:
def polygon(row, col_name):
    coords = []
    for coord in row[col_name]:
        coords.append([coord['lng'], coord['lat']])
    polygon = Polygon(coords)
    
    return polygon

In [124]:
df.head(2)

,id,name,gender,restrictedZone,lng,type,lat,zone,secondaryZone,girlsZone
0,44972,Donburn Primary School,CoEd,False,145.164221,Primary,-37.787439,"[{'lng': 145.1536812449949, 'lat': -37.7890853...",NaN,NaN
1,44971,Sherbourne Primary School,CoEd,False,145.130138,Primary,-37.709067,"[{'lng': 145.13042032578937, 'lat': -37.698348...",NaN,NaN


In [126]:
df['zone_geo'] = df.apply(polygon, col_name='zone', axis=1)

In [127]:
gdf = gpd.GeoDataFrame(df, geometry=df['zone_geo'], crs='4236')

In [128]:
gdf.columns

Index(['id', 'name', 'gender', 'restrictedZone', 'lng', 'type', 'lat', 'zone',
       'secondaryZone', 'girlsZone', 'zone_geo', 'geometry'],
      dtype='object')

In [130]:
gdf = gdf.drop(columns = ['secondaryZone', 'girlsZone', 'zone_geo', 'zone'])

In [131]:
schema = [{'name' : col_name, 'type' : "GEOGRAPHY" if col_name == "geometry" else type_dict.get(col_type.kind, 'STRING')} for (col_name, col_type) in gdf.dtypes.items()]

In [133]:
gdf.to_gbq('melbourne_real_estate.melbourne_primary_school_zones_geo', project_id='robin-sandpit', table_schema=schema)

##### 3.2 secondary schools

In [149]:
sdf = df.loc[df['secondaryZone'].notnull()].copy()

In [150]:
sdf['zone_geo'] = sdf.apply(polygon, col_name='secondaryZone', axis=1)

In [151]:
gdf = gpd.GeoDataFrame(sdf, geometry=sdf['zone_geo'], crs='4236')

In [152]:
gdf = gdf.drop(columns=['zone', 'secondaryZone', 'girlsZone', 'zone_geo'])

In [153]:
schema = [{'name' : col_name, 'type' : "GEOGRAPHY" if col_name == "geometry" else type_dict.get(col_type.kind, 'STRING')} for (col_name, col_type) in gdf.dtypes.items()]

In [155]:
gdf.to_gbq('melbourne_real_estate.melbourne_secondary_school_zones_geo', project_id='robin-sandpit', table_schema=schema, if_exists='replace')

##### 3.3 girls schools

In [147]:
girls = df.loc[df['girlsZone'].notnull()].copy()

In [148]:
girls['zone_geo'] = girls.apply(polygon, col_name='girlsZone', axis=1)

In [156]:
gdf = gpd.GeoDataFrame(girls, geometry=girls['zone_geo'], crs='4236')

In [159]:
gdf = gdf.drop(columns=['zone', 'secondaryZone', 'girlsZone', 'zone_geo'])

In [160]:
schema = [{'name' : col_name, 'type' : "GEOGRAPHY" if col_name == "geometry" else type_dict.get(col_type.kind, 'STRING')} for (col_name, col_type) in gdf.dtypes.items()]

In [161]:
gdf.to_gbq('melbourne_real_estate.melbourne_girls_school_zones_geo', project_id='robin-sandpit', table_schema=schema, if_exists='replace')

### 4. Upload BQ table where field data type is NUMERIC or BIGNUMERIC

In [164]:
# Convert float to decimal
context = decimal.Context(prec=7)
df['work'] = df['work'].apply(context.create_decimal_from_float)